In [ ]:
import numpy as np

In [ ]:
Pauli = [np.array([[1, 0], [0, 1]], dtype = np.complex64), # I
         np.array([[0, 1], [1, 0]], dtype = np.complex64), # X
         np.array([[0, -1j], [1j, 0]], dtype = np.complex64), # Y
         np.array([[1, 0], [0, -1]], dtype = np.complex64)] # Z

In [ ]:
Pauli_names = ["I", "X", "Y", "Z"]

In [ ]:
def Pauli_reversed(matrix):
  for i in range(4):
    if np.linalg.norm(matrix - Pauli[i]) < 1e-6:
      return i, 0
    elif np.linalg.norm(matrix + Pauli[i]) < 1e-6:
      return i, 1

In [ ]:
Pauli2 = [[np.kron(Pauli[i], Pauli[j]) for j in range(4)] for i in range(4)]

In [ ]:
def Pauli2_reversed(matrix):
  for i in range(4):
    for j in range(4):
      if np.linalg.norm(matrix - Pauli2[i][j]) < 1e-6:
        return i, j, 0
      elif np.linalg.norm(matrix + Pauli2[i][j]) < 1e-6:
        return i, j, 1

In [ ]:
Clifford = {"H": np.array([[1, 1], [1, -1]], dtype = np.complex64) / np.sqrt(2),
            "S": np.array([[1, 0], [0, 1j]], dtype = np.complex64),
            "CNOT": np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype = np.complex64)}

In [ ]:
class Clifford_State:

  def __init__(self, n_qubit):
    self.n = n_qubit
    self.state = np.array([[3 if j == i else 0 for j in range(self.n)] for i in range(self.n)]) # this is not the optimal representation, but it is still quadratic in n
    self.sign = np.array([0]*self.n)

  def apply(self, gate, qubits):

    if len(qubits) == 1:
      for q in range(self.n):
        matrix = Clifford[gate] @ Pauli[self.state[q, qubits[0]]] @ np.conjugate(Clifford[gate]).T #np.conjugate compute complex conjugation elt-wise
        self.state[q, qubits[0]], sign_addition = Pauli_reversed(matrix)
        self.sign = (self.sign + sign_addition) % 2

    elif len(qubits) == 2:
      for q in range(self.n):
        matrix = Clifford[gate] @ Pauli2[self.state[q, qubits[0]]][self.state[q, qubits[1]]] @ np.conjugate(Clifford[gate]).T
        self.state[q, qubits[0]], self.state[q, qubits[1]], sign_addition = Pauli2_reversed(matrix)
        self.sign = (self.sign + sign_addition) % 2

  def __repr__(self):
    text = ""
    print(self.sign)

    for i in range(self.n):
      text += "\n" + str(i) + ": "

      if self.sign[i] == 0:
        text += "+  "
      else:
        text += "-  "
      for j in range(self.n):

        text += Pauli_names[self.state[i,j]] + "  "

    return text

In [ ]:
class Clifford_Circuit:

  def __init__(self, n_qubit):
    self.n = n_qubit
    self.gates = []

  def add(self, gate, qubits):
    self.gates.append((gate, qubits))

  def simulate(self):
    state = Clifford_State(self.n)
    for gate, qubits in self.gates:
      state.apply(gate, qubits)
    return state

**Test with One Qubit**

In [ ]:
circuit = Clifford_Circuit(1)
circuit.add("H", [0])
state = circuit.simulate()
state

[0]



0: +  X  

In [ ]:
circuit = Clifford_Circuit(1)
circuit.add("S", [0])
state = circuit.simulate()
state

[0]



0: +  Z  

**Test with Two Qubits**

In [ ]:
circuit = Clifford_Circuit(2)
circuit.add("H", [0])
circuit.add("CNOT", [0, 1])
state = circuit.simulate()
state

[0 0]



0: +  X  X  
1: +  Z  Z  